# Import package

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Mount Drive

In [0]:
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from os.path import join


#project path: content/drive/My Drive/ca675_CT_assiment2_share
PROJ = "My Drive/ca675_CT_assiment2_share"
PROJECT_PATH = join(ROOT, PROJ)


DATASET = "yelp_academic_dataset"
DATASET_PATH = join(PROJECT_PATH, DATASET)


#content/drive/My Drive/ca675_CT_assiment2_private/private_output
PRIVATE_PATH = join(ROOT,"My Drive/ca675_CT_assiment2_private/private_output")

!ls '{DATASET_PATH}'
!ls '{PRIVATE_PATH }'

Dataset_Challenge_Dataset_Agreement.pdf  yelp_academic_dataset_review.json
yelp_academic_dataset_business.json	 yelp_academic_dataset_tip.json
yelp_academic_dataset_checkin.json	 yelp_academic_dataset_user.json
 city_info.csv	    'Copy of city_info_bak.csv'   restaurants_v2.csv
 city_info_new.csv  'Copy of city_info.gsheet'	  restaurants_with_tips_v2.csv


# S1. Import Data

In [0]:
yelp_academic_dataset_business = pd.read_json(DATASET_PATH + '/yelp_academic_dataset_business.json',lines=True)
yelp_academic_dataset_tip = pd.read_json(DATASET_PATH + '/yelp_academic_dataset_tip.json',lines=True)

# S2: Select out restaurants entries
-  filter not null categories from the business datasets

- Why drop the null categories:
 - lenId=192609
 - lenNullCat=482
 - category_miss_ratio=0.0025024791157214877
- missing ratio is very low, so we drop the entries whose "categoreis" is null.

In [0]:
# category_miss_ratio
lenId = len(set(yelp_academic_dataset_business['business_id']))
lenNullCat = yelp_academic_dataset_business['categories'].isnull().sum()
print(' lenId={}\n lenlenNullCat={}\n category_miss_ratio={}'.format(lenId,lenNullCat,lenNullCat/lenId))


 lenId=192609
 lenlenNullCat=482
 category_miss_ratio=0.0025024791157214877


In [0]:
df_business = yelp_academic_dataset_business[yelp_academic_dataset_business['categories'].notnull()]
df_restaurants = df_business[df_business['categories'].str.contains('Restaurants',case=False)]

# S3. Clean Data

In [0]:
# clean city name spell error 
city_info = df_restaurants.loc[:,['city','latitude','longitude','state']]

# Format the uppercase and lowercase
city_info.loc[:,'city'] = city_info.loc[:,'city'].str.title()

# sort value for easier check the spell error
city_info= city_info.sort_values(by='city')

# save city info to drive，spell check by google sheet tool.
#city_info.to_csv(join(PRIVATE_PATH,'city_info.csv'))

# reload the city_info_new to drive
#content/drive/My Drive/ca675_CT_assiment2_private/private_output/city_info_new.csv
city_info_new = pd.read_csv(join(PRIVATE_PATH,'city_info_new.csv'),index_col=0,header=None,names=['city', 'latitude', 'longitude', 'state'])

# merge the df_restaurants with the new city name df
tmp_res_mg_city = pd.merge(left=df_restaurants,right=city_info_new,how='inner',left_index=True,right_index=True)

# replace the old city name with new
tmp_res_mg_city.loc[:,'city_x'] = tmp_res_mg_city.loc[:,'city_y']

# drop unrelevant columns
tmp_res_mg_city.drop(labels=['state_x', 'city_y', 'latitude_y','longitude_y', 'state_y'], axis='columns',inplace=True)

# rename the column names
rename_mapper = {'city_x':'city','latitude_x':'latitude','longitude_x':'longitude'}
tmp_res_mg_city.rename(mapper=rename_mapper,axis = 'columns',inplace=True)

# format the index
tmp_res_mg_city.index = tmp_res_mg_city.index.astype(np.int64)

# check the result
tmp_res_mg_city.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0
13,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-On-The-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0
17,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,None,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0


# S4. Integrate Data

In [0]:
df_restaurants_new = tmp_res_mg_city
restaurants_mg_tips = pd.merge(df_restaurants_new, yelp_academic_dataset_tip,how='left',on='business_id')
restaurants_mg_tips.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,compliment_count,date,text,user_id
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,0.0,2016-08-08 01:48:14,"Spring & egg rolls are great, get them with ex...",fENhF1F6I9c2ssnkfprYzw
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,0.0,2013-06-21 23:44:22,Vile. Worst meal I've had in years. Absolute...,RscJR5zqb3WL4eQezBX5rA
2,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,0.0,2017-10-07 00:56:30,The crab claw balls and sticky rice are amazing!,nyl_1VcRIAyI55bb_scpdw
3,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,1.0,2016-08-21 00:01:11,"If you don't mind their noisy attitude, this i...",fENhF1F6I9c2ssnkfprYzw
4,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,0.0,2014-03-29 00:03:16,"They warmed up our dim sum before serving, wha...",ci8R1Bg83hNwXPyHAEzAFA


# S5. Save Result

In [0]:
#restaurants_mg_tips.to_csv(join(PRIVATE_PATH,'restaurants_with_tips_v2.csv'),index=False)